In [1]:
# =========================
# 0. 라이브러리 / 재현성 설정
# =========================
import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)

# =========================
# 1. 데이터 정의 (Bipolar OR 형태)
# =========================
# x: (4, 2)  -> 입력 2개
# y: (4, 1)  -> 출력 1개, -1 또는 +1
x = np.array([[0.0, 0.0],
              [0.0, 1.0],
              [1.0, 0.0],
              [1.0, 1.0]], dtype=np.float32)

y = np.array([[-1.0],
              [ 1.0],
              [ 1.0],
              [ 1.0]], dtype=np.float32)

n_input = 2
n_output = 1

# =========================
# 2. 모델(단층 퍼셉트론) 구성
# =========================
perceptron = tf.keras.Sequential([
    tf.keras.layers.Dense(
        units=n_output,
        activation="tanh",       # 출력 범위: (-1, +1) -> y와 궁합이 좋음
        input_shape=(n_input,),  # (2,) 로 정확히 지정
        kernel_initializer="random_uniform",
        bias_initializer="zeros"
    )
])

# =========================
# 3. 컴파일(학습 설정)
# =========================
perceptron.compile(
    loss="mse",
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
    metrics=["mse"]
)

# =========================
# 4. 학습
# =========================
history = perceptron.fit(
    x, y,
    epochs=500,
    verbose=0  # 0: 조용히, 1: 진행바, 2: epoch 로그
)

# =========================
# 5. 예측
# =========================
res = perceptron.predict(x, verbose=0)

print("=== Raw prediction (tanh output) ===")
print(res)

# tanh 출력값을 -1 / +1 클래스로 변환 (0 기준으로 부호 판단)
pred_class = np.where(res >= 0, 1.0, -1.0)

print("\n=== Class prediction (-1 or +1) ===")
print(pred_class)

print("\n=== Ground truth (y) ===")
print(y)

# =========================
# 6. 학습된 가중치/바이어스 확인
# =========================
W, b = perceptron.layers[0].get_weights()
print("\n=== Learned weights (W) and bias (b) ===")
print("W:\n", W)



/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


=== Raw prediction (tanh output) ===
[[-0.8164045 ]
 [ 0.8864201 ]
 [ 0.88641614]
 [ 0.99926734]]

=== Class prediction (-1 or +1) ===
[[-1.]
 [ 1.]
 [ 1.]
 [ 1.]]

=== Ground truth (y) ===
[[-1.]
 [ 1.]
 [ 1.]
 [ 1.]]

=== Learned weights (W) and bias (b) ===
W:
 [[2.5508857]
 [2.5509043]]


In [2]:
import torch
import torch.nn as nn

# =========================
# 1. 재현성(옵션이지만 권장)
# =========================
torch.manual_seed(42)

# =========================
# 2. 데이터 (float32)
# =========================
x = torch.tensor([[0.0, 0.0],
                  [0.0, 1.0],
                  [1.0, 0.0],
                  [1.0, 1.0]], dtype=torch.float32)

y = torch.tensor([[-1.0],
                  [ 1.0],
                  [ 1.0],
                  [ 1.0]], dtype=torch.float32)

# =========================
# 3. 모델: Linear(2->1) + tanh
# =========================
model = nn.Sequential(
    nn.Linear(2, 1),
    nn.Tanh()
)

# (TF 코드와 맞추려면 bias=0으로 시작하는 게 가장 유사)
with torch.no_grad():
    model[0].bias.zero_()

# =========================
# 4. 손실/옵티마이저
# =========================
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# =========================
# 5. 학습(epochs=500)
# =========================
for epoch in range(500):
    pred = model(x)                 # forward
    loss = criterion(pred, y)       # loss

    optimizer.zero_grad()           # grad 초기화
    loss.backward()                 # backward
    optimizer.step()                # update

# =========================
# 6. 예측
# =========================
with torch.no_grad():
    res = model(x)
    pred_class = torch.where(res >= 0, torch.tensor(1.0), torch.tensor(-1.0))

print("=== Raw prediction (tanh output) ===")
print(res)

print("\n=== Class prediction (-1 or +1) ===")
print(pred_class)

print("\n=== Ground truth (y) ===")
print(y)

# =========================
# 7. 학습된 파라미터 확인
# =========================
W = model[0].weight.data   # shape (1,2)
b = model[0].bias.data     # shape (1,)
print("\n=== Learned weights (W) and bias (b) ===")
print("W:\n", W)
print("b:\n", b)

# =========================
# 8. 부호 기준 정확도
# =========================
acc = (pred_class == y).float().mean().item()
print(f"\nSign-based accuracy: {acc*100:.2f}%")


=== Raw prediction (tanh output) ===
tensor([[-0.8196],
        [ 0.8884],
        [ 0.8884],
        [ 0.9993]])

=== Class prediction (-1 or +1) ===
tensor([[-1.],
        [ 1.],
        [ 1.],
        [ 1.]])

=== Ground truth (y) ===
tensor([[-1.],
        [ 1.],
        [ 1.],
        [ 1.]])

=== Learned weights (W) and bias (b) ===
W:
 tensor([[2.5699, 2.5699]])
b:
 tensor([-1.1557])

Sign-based accuracy: 100.00%
